In [1]:
import cv2
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
def predict_image(train_im, temp_im):
    temp_im = temp_im
    train_im = cv2.GaussianBlur(train_im, (3,3), sigmaX=0)
    train_im = cv2.GaussianBlur(train_im, (3,3), sigmaX=0)
    orb = cv2.ORB_create(50000, 1.05, nlevels=20, edgeThreshold = 8, patchSize = 7)
    point_k1, desc1 = orb.detectAndCompute(temp_im, None) 
    point_k2, desc2 = orb.detectAndCompute(train_im, None) 
    fea_b = cv2.BFMatcher()
    best_match = fea_b.knnMatch(desc1, desc2, k=2) 
    all_best_m = []
    for m,n in best_match:
        if m.distance < 0.65*n.distance:
            all_best_m.append([m])
    all_im = cv2.drawMatchesKnn(temp_im, point_k1, train_im, point_k2, all_best_m, None, flags = 2) 
    impk1 = cv2.drawKeypoints(temp_im, point_k1, None) 
    impk2 = cv2.drawKeypoints(train_im, point_k2, None)     
    bbox = [] 
    yolo_result = []
    minl = []
    orb = cv2.ORB_create(50000, 1.05, nlevels=20, edgeThreshold = 8, patchSize = 7)
    point_k1, desc1 = orb.detectAndCompute(temp_im, None)
    point_k2, desc2 = orb.detectAndCompute(train_im, None)
    x = np.array([point_k2[0].pt])
    for i in range(len(point_k2)):
        x = np.append(x, [point_k2[i].pt], axis=0)
    x = x[1:len(x)]
    bandwidth = estimate_bandwidth(x, quantile=0.0099)#, n_samples=100000)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=True)
    ms.fit(x)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    s = [None] * n_clusters_
    for i in range(n_clusters_):
        l = ms.labels_
        d, = np.where(l == i)
        # print(d.__len__())
        s[i] = list(point_k2[xx] for xx in d)
    desc2_ = desc2
    all_im = train_im.copy()
    for i in range(n_clusters_):
        point_k2 = s[i]
        l = ms.labels_
        d, = np.where(l == i)
        desc2 = desc2_[d, ]
        flanin = 0
        index_params = dict(algorithm = flanin, trees = 5)
        search_params = dict(checks = 50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        desc1 = np.float32(desc1)
        desc2 = np.float32(desc2)
        try:
          best_match = flann.knnMatch(desc1, desc2, 2)
        except:
          continue
        all_best_m = []
        for m,n in best_match:
            if m.distance < 0.65 * n.distance:
                all_best_m.append(m)
        if len(all_best_m)>5:
            src_pts = np.float32([ point_k1[m.queryIdx].pt for m in all_best_m ]).reshape(-1,1,2)
            dst_pts = np.float32([ point_k2[m.trainIdx].pt for m in all_best_m ]).reshape(-1,1,2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0) # ИСПРАВИЛА ПОСЛ ПАРАМЕТР С 2 НА 5.0
            if M is None:
              continue
            else:
                best_matchMask = mask.ravel().tolist()
                h,w,_ = temp_im.shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)
                dst_piphagorus = np.sqrt((dst[0][0][0] - dst[2][0][0])**2 + (dst[0][0][1] - dst[2][0][1])**2)
                if (dst_piphagorus <= 1.2 * np.sqrt((temp_im.shape[0])**2 + (temp_im.shape[1])**2) ) and (dst_piphagorus >= 0.9 * np.sqrt((temp_im.shape[0])**2 + (temp_im.shape[1])**2) ): 
                  xmin = int(dst[0][0][0])
                  ymin = int(dst[0][0][1])
                  xmax = int(dst[2][0][0])
                  ymax = int(dst[2][0][1])
                  if all(((i[0] - xmin)**2 + (i[1] - ymin)**2) >= (min(temp_im.shape[0]//1.5, temp_im.shape[1]//1.5))**2 for i in minl) or len(minl) == 0:
                    bbox.append([xmin, ymin, xmax, ymax])
                    yolo_result.append((xmin / train_im.shape[0], ymin / train_im.shape[1], (xmax - xmin) / train_im.shape[0], (ymax - ymin) / train_im.shape[1] ) )
                    minl.append((xmin, ymin))
    else:
        best_matchMask = None
    for bbox in bbox:
      all_im = cv2.rectangle(all_im, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255,0,0), 3)
    return yolo_result

In [2]:
img = cv2.imread("train/train_0.jpg")
query = cv2.imread('train/template_0_0.jpg')

In [3]:
predict_image(img, query)

[(0.15625, 0.8347222222222223, 0.0484375, 0.1875),
 (0.15546875, 0.5583333333333333, 0.046875, 0.19166666666666668),
 (0.10546875, 1.101388888888889, 0.04609375, 0.18055555555555555),
 (0.10078125, 0.5402777777777777, 0.05, 0.2013888888888889),
 (0.08828125, 0.21388888888888888, 0.05390625, 0.22361111111111112),
 (0.15078125, 1.3291666666666666, 0.0515625, 0.20972222222222223),
 (0.14453125, 0.22361111111111112, 0.053125, 0.21944444444444444),
 (0.090625, 1.3277777777777777, 0.0546875, 0.2125),
 (0.0890625, 1.6083333333333334, 0.0546875, 0.19583333333333333)]